# Performance 3

In [ ]:
# known import statements
import pandas as pd
import csv
from subprocess import check_output

# new import statements
import zipfile
from io import TextIOWrapper

### Let's take a look at the files inside the current working directory.

In [ ]:
str(check_output(["ls", "-lh"]), encoding="utf-8").split("\n")

### Let's `unzip` "wi.zip".

In [ ]:
check_output(["unzip", "wi.zip"])

### Let's take a look at the files inside the current working directory.

In [ ]:
str(check_output(["ls", "-lh"]), encoding="utf-8").split("\n")

### Traditional way of reading data using pandas

In [ ]:
df = pd.read_csv(???)

memory doesn't increase after this assignment because df2 directly points to the df in memory (it's not making a copy)

In [ ]:
df2 = df

### Clearing memory using re-assignment.
In python, you can clear memory used up for an object simply by getting rid of all the active references. A garabage collector will clean up any inactive references. 

In [ ]:
df = None

In [ ]:
df2 = 123

In [ ]:
df = pd.read_csv("wi.csv") # reload the dataframe
df.head(5) # Top 5 rows within the DataFrame

### How can we see all the column names?

In [ ]:
df.columns

### How to extract `interest_rate`?

In [ ]:
df["interest_rate"] # observe that there are missing values

### How to count unique values in a column `Series`?

In [ ]:
df["interest_rate"].???()

### Let's eliminiate the strings (Exempt) and missing values (NaN).
Let's try `pd.to_numeric(...)`. We need a way to specify that strings need to be converted into NaN values.

In [ ]:
pd.???(df["interest_rate"]) 
# TODO: open the documentation and figure out what parameter will help us

In [ ]:
pd.to_numeric(df["interest_rate"], ???)

### Let's drop the NaN values and compute average interest rate.

In [ ]:
pd.to_numeric(df["interest_rate"], errors="coerce").???()

In [ ]:
pd.to_numeric(df["interest_rate"], errors="coerce").dropna().???()

Re-assignment does not work here because we used "df" to perform other operations, so there are more than one active references. In fact, we don't even have access to some of the active references. In that case, you can only free up the memory after you "shutdown" the current notebook.

In [ ]:
df = "some string"

### How can we read the data without creating an uncompressed version of "wi.csv"?

- Why would we want to do something like that?
    1. lower memory usage (we can try to load data one row at a time, instead of all rows at a time): that will still work for average interest rate computation
    2. lower storage usage (you can directly work with compressed data)
    
**IMPORTANT**: do not run this cell code unless you shutdown the notebook - your kernel will crash (you will run out of memory space)

In [ ]:
# IMPORTANT: do not run this cell code unless you shutdown the notebook - your kernel will crash (you will run out of memory space)
f = open("wi.csv")
# instead of passing relative path of file name, we can pass a file object instance reference
df = pd.read_csv(f) 
f.close()

### Let's free up memory and delete "wi.csv".

In [ ]:
df = "some string"

In [ ]:
check_output(["rm", "wi.csv"])
str(check_output(["ls", "-lh"]), encoding="utf-8").split("\n")

### How can we read data directly from a zip file?
`zipfile.ZipFile(...)`

### Goals:
1. directly access the data without decompressing: `zipfile.ZipFile(...)` - saves storage space by directly opening a zip file
2. only look at one row at a time: `csv.DictReader(...)` - saves memory space by enabling us to read one row at a time (as `dict`)

In [ ]:
# code to achieve goal 1
zf = ???("wi.zip")
f = zf.???("wi.csv")

df = pd.read_csv(f) 

f.close()
zf.close()

In [ ]:
# Free up the memory again
df = "some string"

In [ ]:
# code to achieve goal 1 & goal 2
zf = zipfile.ZipFile("wi.zip")
f = zf.open("wi.csv")

reader = ???(f)

for row in reader:
    print(row)
    break

f.close()
zf.close()

### Let's learn more modes for `open` built-in function
- `open(..., mode="r")`   => text (default)
- `open(..., mode="rb")`  => bytes
- `zf.open(...)`          => always bytes

With `zipfile` module there isn't a way for us to specify that we need text.

### `TextIOWrapper` inside `io` module enables us to convert `bytes` into `str`

In [ ]:
# code for goal 2 & goal 1
zf = zipfile.ZipFile("wi.zip")
f = zf.open("wi.csv")

reader = csv.DictReader(???(f))

for row in reader:
    print(row)
    break

f.close()
zf.close()

### Let's go back to calculating average interest rate.
- Algorithm / Pseudocode steps:
    1. print "interest rate" and type of "interest rate"
    2. convert "interest rate" into `float` - how can we handle errors? `try` ... `except` ... (*IMPORTANT*: always have your `except` block catch specific exceptions)
    3. calculate running total, count for each row of data
    4. calculate average

In [ ]:
zf = zipfile.ZipFile("wi.zip")
f = zf.open("wi.csv")

total = 0
count = 0

reader = csv.DictReader(TextIOWrapper(f))

for row in reader:
    try:
        total += float(row["interest_rate"])
        count += 1
    except ValueError:
        pass # do nothing

f.close()
zf.close()

total / count

### Let's generalize the code to read "interest rate" into a function.

- This does make things worse because we are going back to reading all the data before doing the computation.
- But this sets us up to learn about generators.

In [ ]:
def get_rates_v1():
    pass

rates = get_rates_v1()
sum(rates) / len(rates)

### Using a generator
- `yield` each value
- use `next` to get the next value => internally `for` loop invokes `next` for each iteration

In [ ]:
def get_rates_v2():
    pass

rates = get_rates_v2()

In [ ]:
next(rates) # gives us the next value

In [ ]:
next(rates) # gives us the next value

In [ ]:
next(rates) # gives us the next value

### Let's use `for` loop to keep getting all the rates.

- `len` function doesn't work with generators
- indexing doesn't work with generators

In [ ]:
len(rates)

In [ ]:
rates[4]

In [ ]:
rates = get_rates_v2()

total = 0
count = 0

for rate in rates: # keeps calling next(rates) to get values from yield
    total += rate
    count += 1
    
total / count

This approach doesn't work for median calculation. Why? Remember we have to sort, so we need all values in memory.

In [ ]:
rates = list(get_rates_v2())
rates.sort()